In [1]:
import pandas as pd 
import numpy as np 

pd.set_option('display.max_columns', None)

In [7]:
df = pd.read_csv('../data/raw/occurrence.txt', sep='\t')
df.describe()

C:\Users\SABIO\AppData\Local\Temp\ipykernel_11920\1595757217.py:1: DtypeWarning: Columns (29,30,35,43,46,49,53,60,75,77,83,102,103,112,113,114,115,116,117,118,132,133,137,138,147,170,177) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/raw/occurrence.txt', sep='\t')


,gbifID,publisher,dataGeneralizations,individualCount,organismQuantity,organismQuantityType,caste,behavior,vitality,degreeOfEstablishment,pathway,organismScope,associatedOrganisms,materialEntityID,materialEntityRemarks,verbatimLabel,eventID,parentEventID,startDayOfYear,endDayOfYear,year,month,day,sampleSizeValue,sampleSizeUnit,samplingEffort,higherGeographyID,waterBody,islandGroup,verticalDatum,verbatimDepth,minimumDistanceAboveSurfaceInMeters,maximumDistanceAboveSurfaceInMeters,decimalLatitude,decimalLongitude,coordinateUncertaintyInMeters,coordinatePrecision,pointRadiusSpatialFit,footprintWKT,footprintSRS,footprintSpatialFit,earliestEpochOrLowestSeries,latestEpochOrHighestSeries,earliestAgeOrLowestStage,latestAgeOrHighestStage,lowestBiostratigraphicZone,highestBiostratigraphicZone,lithostratigraphicTerms,group,formation,member,bed,identificationID,verbatimIdentification,acceptedNameUsageID,parentNameUsageID,originalNameUsageID,nameAccordingToID,namePublishedInID,originalNameUsage,nameAccordingTo,namePublishedIn,namePublishedInYear,superfamily,subfamily,tribe,subtribe,subgenus,infragenericEpithet,cultivarEpithet,verbatimTaxonRank,elevation,elevationAccuracy,depth,depthAccuracy,distanceFromCentroidInMeters,taxonKey,acceptedTaxonKey,kingdomKey,phylumKey,classKey,orderKey,familyKey,genusKey,subgenusKey,speciesKey,typifiedName,relativeOrganismQuantity
count,7.240000e+03,0.0,0.0,245.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6441.000000,6441.000000,6822.000000,6542.000000,6436.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28.00000,28.00000,6068.000000,6068.000000,2.524000e+03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.021000e+03,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,455.000000,121.000000,0.0,0.0,0.0,7.240000e+03,7240.0,7240.0,7240.0,7240.0,7240.0,7240.0,7240.0,0.0,7240.0,0.0,0.0
mean,2.726149e+09,NaN,NaN,1.004082,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,205.479429,205.504114,1991.488713,7.273617,15.588564,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1244.51000,1244.51000,47.160052,-118.825543,2.649493e+03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.419873e+08,NaN,168699911.0,168057998.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1654.137978,28.582645,NaN,NaN,NaN,8.629005e+06,9060377.0,6.0,7707728.0,220.0,1353.0,2505.0,2882813.0,NaN,9060377.0,NaN,NaN
std,1.607010e+09,NaN,NaN,0.063888,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.140847,34.066258,35.588496,1.148387,8.836163,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,413.33417,413.33417,3.030156,5.857413,4.860094e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.701348e+08,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,605.732707,65.880521,NaN,NaN,NaN,1.555555e+06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,NaN
min,4.185537e+07,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,19.000000,1858.000000,1.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,560.00000,560.00000,38.310560,-160.014600,3.600000e-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.824340e+05,NaN,168699911.0,168057998.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN,2.882876e+06,9060377.0,6.0,7707728.0,220.0,1353.0,2505.0,2882813.0,NaN,9060377.0,NaN,NaN
25%,1.302779e+09,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,183.000000,183.000000,1968.000000,7.000000,8.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,932.88000,932.88000,44.906400,-122.099200,5.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.984348e+08,NaN,168699911.0,168057998.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1299.000000,0.000000,NaN,NaN,NaN,9.060377e+06,9060377.0,6.0,7707728.0,220.0,1353.0,2505.0,2882813.0,NaN,9060377.0,NaN,NaN
50%,1.949566e+09,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,206.000000,206.000000,2002.

In [10]:
# Select relevant columns for species, time and location
df_filtered = df[['scientificName', 'eventDate', 'decimalLatitude', 'decimalLongitude', 'habitat', 'verbatimElevation']]

# Drop any rows with missing values
#df_filtered = df_filtered.dropna()

# Display first few rows to verify the data
print("\nFiltered dataset with species, time and location:")
df_filtered.head()


Filtered dataset with species, time and location:


,scientificName,eventDate,decimalLatitude,decimalLongitude,habitat,verbatimElevation
0,Vaccinium membranaceum Douglas ex Hook.,NaN,NaN,NaN,NaN,NaN
1,Vaccinium membranaceum Douglas ex Hook.,NaN,NaN,NaN,NaN,NaN
2,Vaccinium membranaceum Douglas ex Hook.,1970-07-11,49.133333,-114.15,"scree slope, facing east",7000 feet
3,Vaccinium membranaceum Douglas ex Hook.,1968-08-04,NaN,NaN,Trail side,NaN
4,Vaccinium membranaceum Douglas ex Hook.,1970-07-11,49.133333,-114.15,"scree slope, facing east",7000 feet


In [27]:
# Convert eventDate to datetime, handling multiple formats
df_filtered['eventDate'] = pd.to_datetime(df_filtered['eventDate'], format='mixed', errors='coerce')

df_filtered_1 = df_filtered.copy()
df_filtered_1.dropna(inplace=True)
df_filtered_1['eventDate'] = pd.to_datetime(df_filtered_1['eventDate'], format='%Y-%m-%d')

df_filtered_1.head().sort_values(by='eventDate', ascending=True)

C:\Users\SABIO\AppData\Local\Temp\ipykernel_11920\1656432752.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['eventDate'] = pd.to_datetime(df_filtered['eventDate'], format='mixed', errors='coerce')


,scientificName,eventDate,decimalLatitude,decimalLongitude,habitat,verbatimElevation
7,Vaccinium membranaceum Douglas ex Hook.,1970-06-23,49.033333,-113.916667,"P. contorta forest, in damp humus",5500 feet
2,Vaccinium membranaceum Douglas ex Hook.,1970-07-11,49.133333,-114.150000,"scree slope, facing east",7000 feet
4,Vaccinium membranaceum Douglas ex Hook.,1970-07-11,49.133333,-114.150000,"scree slope, facing east",7000 feet
6,Vaccinium membranaceum Douglas ex Hook.,1970-07-11,49.133333,-114.150000,"scree slope, facing east",7000 feet
8,Vaccinium membranaceum Douglas ex Hook.,1970-07-11,49.133333,-114.150000,"scree slope, facing east",7000 feet
